In [ ]:
import datetime
import lxml.etree as et
import os

from article_class import Article
from samples.corpus_analysis import get_random_list_of_dois

# Main directory of article XML files
corpusdir = 'allofplos_xml'

# Temp folder for downloading and processing new articles
newarticledir = 'new_plos_articles'

# URL bases for PLOS's raw article XML
EXT_URL_TMP = 'http://journals.plos.org/plosone/article/file?id={0}&type=manuscript'
INT_URL_TMP = 'http://contentrepo.plos.org:8002/v1/objects/mogilefs-prod-repo?key={0}.XML'
URL_TMP = EXT_URL_TMP


In [ ]:
dois = get_random_list_of_dois(count=10000)
for doi in dois:
    article = Article(doi)
    article_data = []
    if article.type_ not in ('correction', 'retraction') and article.pubdate > datetime.datetime(2017, 10, 1):
        article_data = (article.author, article.title, article.journal, article.doi, article.page, article.get_pubdate(string_=True))
        print(article_data)